In [15]:
import pandas as pd
import numpy as np

import requests
import datetime

import yfinance as yf

from alpaca.trading.client import TradingClient
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.requests import MarketOrderRequest, StopLossRequest
from alpaca.trading.stream import TradingStream

import config
import reversal_bot

In [25]:
msft = yf.Ticker("MSFT")
hist = msft.history(period='305m', interval="5m")

print(hist)

                                 Open        High         Low       Close   
Datetime                                                                    
2023-07-28 15:20:00-04:00  339.350006  339.379913  338.640015  338.769989  \
2023-07-28 15:25:00-04:00  338.769989  338.950012  338.190002  338.204987   
2023-07-28 15:30:00-04:00  338.230011  338.910004  338.119995  338.732513   
2023-07-28 15:35:00-04:00  338.720001  338.850006  338.100006  338.114990   
2023-07-28 15:40:00-04:00  338.130005  338.320007  337.959991  338.279999   
...                               ...         ...         ...         ...   
2023-07-31 13:35:00-04:00  334.154999  334.670013  334.116913  334.640808   
2023-07-31 13:40:00-04:00  334.640015  334.640015  334.299988  334.429993   
2023-07-31 13:45:00-04:00  334.429993  334.470001  334.119995  334.320007   
2023-07-31 13:50:00-04:00  334.320007  334.450012  334.130005  334.263489   
2023-07-31 13:55:00-04:00  334.239990  334.420013  334.220001  334.339996   

In [26]:
msft = yf.Ticker('MSFT')
ma_df = msft.history(period='305m', interval='5m')

current_ma = ma_df.tail(20)
previous_ma = ma_df.iloc[len(ma_df)-2: len(ma_df)-22: -1]

In [27]:
previous_ma

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-07-31 13:50:00-04:00,334.320007,334.450012,334.130005,334.263489,144441,0.0,0.0
2023-07-31 13:45:00-04:00,334.429993,334.470001,334.119995,334.320007,168495,0.0,0.0
2023-07-31 13:40:00-04:00,334.640015,334.640015,334.299988,334.429993,139793,0.0,0.0
2023-07-31 13:35:00-04:00,334.154999,334.670013,334.116913,334.640808,197736,0.0,0.0
2023-07-31 13:30:00-04:00,334.079987,334.214996,333.989990,334.149994,181951,0.0,0.0
2023-07-31 13:25:00-04:00,334.170013,334.350006,334.029999,334.100006,156155,0.0,0.0
2023-07-31 13:20:00-04:00,333.929993,334.304993,333.899994,334.184998,150296,0.0,0.0
2023-07-31 13:15:00-04:00,334.445007,334.540009,333.859985,333.950012,220321,0.0,0.0
2023-07-31 13:10:00-04:00,334.660004,334.699890,334.390015,334.445007,154921,0.0,0.0
